In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os.path
import pretty_midi
import pickle
import random

### Evaluate system performance

In [3]:
def readGroundTruthLabels(gtfile):
    d={}
    d1={}
    with open(gtfile, "r") as f:
        for line in f:
            data = line.split(',')
            key = data[0]
            d[key] = []
            d1[key] = key
            for idx,item in enumerate(data):
                if idx != 0 and item != "x":
                    try:
                        pieceNum = int(item)
                        d[key].append(pieceNum)
                    except:
                        pass
                    try:
                        pieceNum = int(item)
                        d1[str(pieceNum)] = key
                    except:
                        pass
    return d, d1

In [4]:
def readHypothesisFiles(hypdir, benchmark):
    l = []
    if benchmark == 1:
        for hypfile in sorted(glob.glob("{}/*.pkl".format(hypdir))):
            with open(hypfile, "rb") as f:
                l.append([os.path.splitext(os.path.basename(hypfile))[0],pickle.load(f)])
    elif benchmark == 0 or benchmark == 2:
        for hypfile in sorted(glob.glob("{}/*.hyp".format(hypdir))):
            print(hypfile)
            with open(hypfile, "rb") as f:
                l.append(pickle.load(f))
    elif benchmark == 3:
        for hypfile in sorted(glob.glob("{}/*.hyp".format(hypdir))):
            print(hypfile)
            with open(hypfile, "rb") as f:
                data = pickle.load(f)
                pieceScores = [(x[1],x[0]) for x in data[1]]
                l.append((data[0],pieceScores))
    return l

In [5]:
s = set([])
def collapseIds(pieceScores):
    pieceScores = list(dict.fromkeys(pieceScores))
    return pieceScores
        

In [6]:
def getRank(pieceScores,gt,idt,queryid,benchmark, condition):
    query = queryid.split('_')[0]
    l = []
    count = 0
    if benchmark:
        pieceScores = collapseIds(pieceScores)
    rank = 300000
    for i in range(len(pieceScores)):
        if benchmark == 1 or benchmark == 2:
            num = pieceScores[i].split("_")[0]
            if 'p' in num:
                if condition == 2:
                    continue
                if query.split("_")[0] == num:
                    rank = count+1
                    return rank
            elif int(num) in gt[query]:
                rank = count+1
                return rank
            count = count+1            
        else:
            if pieceScores[i][0][0]=='p':
                if condition == 2:
                        continue
                if(pieceScores[i][0]==queryid.split('_')[0]):
                    rank=count+1
                    return rank
                if not pieceScores[i][0] in idt.keys():
                    count+=1
                elif not idt[pieceScores[i][0]] in l:
                    count+=1
                    l.append(idt[idt[pieceScores[i][0]]])
            else:
                try:
                    if (int(pieceScores[i][0].split("**")[-1]) in gt[query]):
                        rank = count+1
                        return rank

                    if not str(int(pieceScores[i][0].split("**")[-1])) in idt.keys():
                        count+=1
                    elif not idt[str(int(pieceScores[i][0].split("**")[-1]))] in l:
                        count+=1
                        l.append(idt[str(int(pieceScores[i][0].split("**")[-1]))])
                except:
                    pass
    return rank

In [19]:
def calcPrecisionRecall(hypdir, gtfile, benchmark = False, condition = 1):
    hyps = readHypothesisFiles(hypdir, benchmark)
    gt,idt = readGroundTruthLabels(gtfile)
    MRR = 0
    runtimes = []
    MRRs = []
    if condition == 2:
        valid = []
        with open('condition2.txt','r') as f:
            for line in f:
                valid.append(line.strip())
                
    hyps = sorted(hyps, key = lambda x: int(x[0].split("_")[0][1:])*100+int(x[0].split("_")[1][1:]))
    count = 0
    if hyps.shape[1] == 2:
        for queryid, pieceScores in hyps:
            if condition == 2:
                if not queryid.split('_')[0].strip() in valid:
                    count+=1
                    continue
            rank = getRank(pieceScores,gt,idt,queryid,benchmark, condition)
            MRRs.append((queryid,rank))
            MRR=MRR+1/(rank)
    else:
        for queryid, pieceScores, runtime in hyps:
            if condition == 2:
                if not queryid.split('_')[0].strip() in valid:
                    count+=1
                    continue
            rank = getRank(pieceScores,gt,idt,queryid,benchmark, condition)
            MRRs.append((queryid,rank))
            runtimes.append(runtime)
            MRR=MRR+1/(rank)
    return MRR/len(MRRs),MRRs, runtimes

In [20]:
def calcOverlap(seg1, seg2):
    overlap_lb = max(seg1[0], seg2[0])
    overlap_ub = min(seg1[1], seg2[1])
    overlap = np.clip(overlap_ub - overlap_lb, 0, None)
    return overlap    

In [21]:
hypdir = 'experiments/DynamicStatic1k/hyp'
condition2 = "condition2.txt"

#0 is normal
#1 is old paper
#2 is new paper
#3 is weird test
benchmark = 0
queryGTFile = 'piece_To_id.csv'
MRR, MRRs, runtimes = calcPrecisionRecall(hypdir, queryGTFile, benchmark, condition = 1)

experiments/DynamicStatic1k/hyp/p101_q1.hyp
experiments/DynamicStatic1k/hyp/p101_q10.hyp
experiments/DynamicStatic1k/hyp/p101_q2.hyp
experiments/DynamicStatic1k/hyp/p101_q3.hyp
experiments/DynamicStatic1k/hyp/p101_q4.hyp
experiments/DynamicStatic1k/hyp/p101_q5.hyp
experiments/DynamicStatic1k/hyp/p101_q6.hyp
experiments/DynamicStatic1k/hyp/p101_q7.hyp
experiments/DynamicStatic1k/hyp/p101_q8.hyp
experiments/DynamicStatic1k/hyp/p101_q9.hyp
experiments/DynamicStatic1k/hyp/p105_q1.hyp
experiments/DynamicStatic1k/hyp/p105_q10.hyp
experiments/DynamicStatic1k/hyp/p105_q2.hyp
experiments/DynamicStatic1k/hyp/p105_q3.hyp
experiments/DynamicStatic1k/hyp/p105_q4.hyp
experiments/DynamicStatic1k/hyp/p105_q5.hyp
experiments/DynamicStatic1k/hyp/p105_q6.hyp
experiments/DynamicStatic1k/hyp/p105_q7.hyp
experiments/DynamicStatic1k/hyp/p105_q8.hyp
experiments/DynamicStatic1k/hyp/p105_q9.hyp
experiments/DynamicStatic1k/hyp/p111_q1.hyp
experiments/DynamicStatic1k/hyp/p111_q10.hyp
experiments/DynamicStatic1k/h

experiments/DynamicStatic1k/hyp/p185_q1.hyp
experiments/DynamicStatic1k/hyp/p185_q10.hyp
experiments/DynamicStatic1k/hyp/p185_q2.hyp
experiments/DynamicStatic1k/hyp/p185_q3.hyp
experiments/DynamicStatic1k/hyp/p185_q4.hyp
experiments/DynamicStatic1k/hyp/p185_q5.hyp
experiments/DynamicStatic1k/hyp/p185_q6.hyp
experiments/DynamicStatic1k/hyp/p185_q7.hyp
experiments/DynamicStatic1k/hyp/p185_q8.hyp
experiments/DynamicStatic1k/hyp/p185_q9.hyp
experiments/DynamicStatic1k/hyp/p191_q1.hyp
experiments/DynamicStatic1k/hyp/p191_q10.hyp
experiments/DynamicStatic1k/hyp/p191_q2.hyp
experiments/DynamicStatic1k/hyp/p191_q3.hyp
experiments/DynamicStatic1k/hyp/p191_q4.hyp
experiments/DynamicStatic1k/hyp/p191_q5.hyp
experiments/DynamicStatic1k/hyp/p191_q6.hyp
experiments/DynamicStatic1k/hyp/p191_q7.hyp
experiments/DynamicStatic1k/hyp/p191_q8.hyp
experiments/DynamicStatic1k/hyp/p191_q9.hyp
experiments/DynamicStatic1k/hyp/p195_q1.hyp
experiments/DynamicStatic1k/hyp/p195_q10.hyp
experiments/DynamicStatic1k/h

p1_q1 1
p1_q2 1
p1_q3 1
p1_q4 1
p1_q5 1
p1_q6 1
p1_q7 1
p1_q8 1
p1_q9 1
p1_q10 1
p5_q1 1
p5_q2 1
p5_q3 1
p5_q4 1
p5_q5 1
p5_q6 1
p5_q7 1
p5_q8 1
p5_q9 1
p5_q10 1
p11_q1 1
p11_q2 1
p11_q3 1
p11_q4 1
p11_q5 1
p11_q6 1
p11_q7 1
p11_q8 1
p11_q9 1
p11_q10 1
p15_q1 1
p15_q2 1
p15_q3 1
p15_q4 1
p15_q5 1
p15_q6 1
p15_q7 1
p15_q8 1
p15_q9 1
p15_q10 1
p21_q1 1
p21_q2 1
p21_q3 1
p21_q4 1
p21_q5 300000
p21_q6 1
p21_q7 1
p21_q8 1
p21_q9 1
p21_q10 1
p25_q1 1
p25_q2 1
p25_q3 1
p25_q4 1
p25_q5 1
p25_q6 1
p25_q7 1
p25_q8 1
p25_q9 1
p25_q10 1
p31_q1 1
p31_q2 1
p31_q3 1
p31_q4 1
p31_q5 1
p31_q6 1
p31_q7 1
p31_q8 1
p31_q9 1
p31_q10 1
p35_q1 1
p35_q2 1
p35_q3 1
p35_q4 1
p35_q5 1
p35_q6 1
p35_q7 1
p35_q8 1
p35_q9 1
p35_q10 1
p41_q1 513
p41_q2 1
p41_q3 1
p41_q4 1
p41_q5 1
p41_q6 1
p41_q7 1
p41_q8 1
p41_q9 1
p41_q10 1
p45_q1 1
p45_q2 1
p45_q3 1
p45_q4 1
p45_q5 1
p45_q6 1
p45_q7 1
p45_q8 1
p45_q9 1
p45_q10 1
p51_q1 1
p51_q2 1
p51_q3 1
p51_q4 1
p51_q5 1
p51_q6 1
p51_q7 1
p51_q8 1
p51_q9 1
p51_q10 300000
p55_q1 

In [22]:
MRR

0.9008712772533238

### Investigate Errors

In [23]:
def printDebuggingInfo(MRRs):
    for i, (queryid, rank) in enumerate(MRRs):
        print(queryid, rank)

In [24]:
printDebuggingInfo(MRRs)

p1_q1 1
p1_q2 1
p1_q3 1
p1_q4 1
p1_q5 1
p1_q6 1
p1_q7 1
p1_q8 1
p1_q9 1
p1_q10 1
p5_q1 1
p5_q2 1
p5_q3 1
p5_q4 1
p5_q5 1
p5_q6 1
p5_q7 1
p5_q8 1
p5_q9 1
p5_q10 1
p11_q1 1
p11_q2 1
p11_q3 1
p11_q4 1
p11_q5 1
p11_q6 1
p11_q7 1
p11_q8 1
p11_q9 1
p11_q10 1
p15_q1 1
p15_q2 1
p15_q3 1
p15_q4 1
p15_q5 1
p15_q6 1
p15_q7 1
p15_q8 1
p15_q9 1
p15_q10 1
p21_q1 1
p21_q2 1
p21_q3 1
p21_q4 1
p21_q5 300000
p21_q6 1
p21_q7 1
p21_q8 1
p21_q9 1
p21_q10 1
p25_q1 1
p25_q2 1
p25_q3 1
p25_q4 1
p25_q5 1
p25_q6 1
p25_q7 1
p25_q8 1
p25_q9 1
p25_q10 1
p31_q1 1
p31_q2 1
p31_q3 1
p31_q4 1
p31_q5 1
p31_q6 1
p31_q7 1
p31_q8 1
p31_q9 1
p31_q10 1
p35_q1 1
p35_q2 1
p35_q3 1
p35_q4 1
p35_q5 1
p35_q6 1
p35_q7 1
p35_q8 1
p35_q9 1
p35_q10 1
p41_q1 513
p41_q2 1
p41_q3 1
p41_q4 1
p41_q5 1
p41_q6 1
p41_q7 1
p41_q8 1
p41_q9 1
p41_q10 1
p45_q1 1
p45_q2 1
p45_q3 1
p45_q4 1
p45_q5 1
p45_q6 1
p45_q7 1
p45_q8 1
p45_q9 1
p45_q10 1
p51_q1 1
p51_q2 1
p51_q3 1
p51_q4 1
p51_q5 1
p51_q6 1
p51_q7 1
p51_q8 1
p51_q9 1
p51_q10 300000
p55_q1 

### Measure Runtime

In [22]:
def showRuntimeStats(durs):
    durs = []
    durs = np.array(durs)
    avgDur = np.mean(durs)
    minDur = np.min(durs)
    maxDur = np.max(durs)
    stdDur = np.std(durs)
    print('Avg Duration: {:.2f} sec'.format(avgDur))
    print('Std Duration: {:.2f} sec'.format(stdDur))
    print('Min Duration: {:.2f} sec'.format(minDur))
    print('Max Duration: {:.2f} sec'.format(maxDur))
    plt.hist(durs, bins=np.arange(0,2,.1))
    plt.xlabel('Runtime (sec)')
    plt.ylabel('Count')

In [23]:
showRuntimeStats(runtimes)

ValueError: could not convert string to float: '_S.171_**423369q\x04Ke\x86q\x05X(\x00\x00\x00dLiszt'